In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import warnings
import csv
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import pickle
import random

In [2]:
#Create the data file if it doesn't exist
def createFile():
    try:
        f = open("linkedinProfileData.csv")
    except IOError:
        with open('linkedinProfileData.csv', 'w') as csvFile:
            csvWriter = csv.writer(csvFile)
            csvWriter.writerow(["number","name","headline","about","role","organization","time","location","description","education","degree","major","grade","time","description","licenses and certifications","skills","accomplishments",'interests'])
    else:
        f.close()

In [3]:
#Function to implement infinite scrolling and scrape results
#Saves required data and returns a list with new names added
def scrapeData(driver,url,number,nameList):
    driver.get(url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            random.seed()
            rand = random.randint(1,2)
            if rand==1:
                driver.execute_script("window.scrollTo(0, 0);")
            try:
                
                element_present = EC.presence_of_element_located((By.XPATH, "//h3[@class='pv-entity__position-group-pager pv-profile-section__list-item ember-view']"))
                WebDriverWait(driver, 5).until(element_present)
            except:
                pass
            break
        last_height = new_height
    
    try:
        element = driver.find_element_by_css_selector('button[class*="pv-browsemap-section__show-cta artdeco-card__action artdeco-button artdeco-button--icon-right artdeco-button--2 artdeco-button--fluid artdeco-button--tertiary ember-view"]')
        driver.execute_script("arguments[0].click();", element)
    except:
        pass
    
    try:
        element = driver.find_element_by_css_selector('a[class*="lt-line-clamp__more"]')
        driver.execute_script("arguments[0].click();", element)
    except:
        pass
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    appendData=list()
    appendData.append(number)
   
    name=soup.find_all(class_='inline t-24 t-black t-normal break-words',limit=1)
    name=name[0].getText()
    name=name.strip()
    appendData.append(name)
    
    headline=soup.find_all(class_='mt1 t-18 t-black t-normal break-words',limit=1)
    if not headline:
        appendData.append("NA")
    else:
        headline=headline[0].getText()
        headline=headline.strip()
        appendData.append(headline)
        
    abt=soup.find_all(class_='pv-about__summary-text mt4 t-14 ember-view')
    if abt:
        abtStr=abt[0].getText()
        appendData.append(abtStr[:-13])
    else:
        appendData.append("NA")
    
    educ=soup.find_all(class_='pv-entity__position-group-pager pv-profile-section__list-item ember-view')
    roleList=[]
    orgList=[]
    descList=[]
    timeList=[]
    locList=[]
    if not educ:
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
    else:
        for item in educ:
            role=item.find_all(class_='t-16 t-black t-bold')
            role=role[0].getText()
            if "Company Name" in role:
                role=item.find_all(class_='t-14 t-black t-bold')
                role=role[0].getText()
                role=role.strip()
                role=role[6:]
                org=item.find_all(class_='t-16 t-black t-bold')
                org=org[0].getText()
                org=org.strip()
                org=org[13:]
            else:
                org=item.find_all(class_='pv-entity__secondary-title t-14 t-black t-normal')
                org=org[0].getText()
                org=org.strip()
            desc=item.find_all(class_="pv-entity__description t-14 t-black t-normal inline-show-more-text inline-show-more-text--is-collapsed ember-view")
            if desc:
                desc=desc[0].getText()
                desc=desc[2:]
                desc=desc[:desc.index('\n')]
            else:
                desc="NA"
            timeL=item.find_all(class_="pv-entity__date-range t-14 t-black--light t-normal")
            if timeL:
                timeL=timeL[0].find_all('span')
                timeL=timeL[1].getText()
            else:
                timeL="NA"
            location=item.find_all(class_='pv-entity__location t-14 t-black--light t-normal block',limit=1)
            if location:
                location=location[0].getText()
                location=location.strip()
                location=location[9:]
            else:
                location="NA"    
            timeList.append(timeL)
            roleList.append(role)
            orgList.append(org)
            descList.append(desc)
            locList.append(location)
            
        
        roleStr=""
        for item in roleList:
            roleStr=roleStr+str(item)+' | '
        orgStr=""
        for item in orgList:
            orgStr=orgStr+str(item)+' | '
        descStr=""
        for item in descList:
            descStr=descStr+str(item)+' | '
        timeStr=""
        for item in timeList:
            timeStr=timeStr+str(item)+' | '
        locStr=""
        for item in locList:
            locStr=locStr+str(item)+' | '
        
        appendData.append(roleStr)
        appendData.append(orgStr)
        appendData.append(timeStr)
        appendData.append(locStr)
        appendData.append(descStr)
        
    educ=soup.find_all(class_='pv-profile-section__list-item pv-education-entity pv-profile-section__card-item ember-view')
    schoolList=[]
    degreeList=[]
    majorList=[]
    GPAList=[]
    timeList=[]
    descList=[]
    if not educ:
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
        appendData.append("NA")
    else:
        for item in educ:
            school=item.find_all(class_='pv-entity__school-name t-16 t-black t-bold',limit=1)
            school=school[0].getText()
            school=school.strip()
            degree=item.find_all(class_='pv-entity__secondary-title pv-entity__degree-name t-14 t-black t-normal',limit=1)
            if degree:
                degree=degree[0].getText()
                degree=degree.strip()
                degree=degree[12:]
            else:
                degree='NA'
            major=item.find_all(class_='pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal')
            if major:
                major=major[0].getText()
                major=major.strip()
                major=major[15:]
            else:
                major="NA"
            GPA=item.find_all(class_='pv-entity__secondary-title pv-entity__grade t-14 t-black t-normal')
            if GPA:
                GPA=GPA[0].getText()
                GPA=GPA.strip()
                GPA=GPA[6:]
            else:
                GPA="NA"
            timeL=item.find_all(class_='pv-entity__dates t-14 t-black--light t-normal')
            if timeL:
                timeL=timeL[0].getText()
                timeL=timeL.strip()
                timeL=timeL[38:]
            else:
                timeL="NA"
            desc=item.find_all(class_='pv-entity__description t-14 t-normal mt4')
            if desc:
                desc=desc[0].getText()
            else:
                desc="NA"
            schoolList.append(school)
            degreeList.append(degree)
            majorList.append(major)
            GPAList.append(GPA)
            timeList.append(timeL)
            descList.append(desc)
            
        schoolStr=""
        for item in schoolList:
            schoolStr=schoolStr+str(item)+' | '
        degreeStr=""
        for item in degreeList:
            degreeStr=degreeStr+str(item)+' | '
        majorStr=""
        for item in majorList:
            majorStr=majorStr+str(item)+' | '
        GPAStr=""
        for item in GPAList:
            GPAStr=GPAStr+str(item)+' | '
        timeStr=""
        for item in timeList:
            timeStr=timeStr+str(item)+' | '
        descStr=""
        for item in descList:
            descStr=descStr+str(item)+' | '
            
        appendData.append(schoolStr)
        appendData.append(degreeStr)
        appendData.append(majorStr)
        appendData.append(GPAStr)
        appendData.append(timeStr)
        appendData.append(descStr)
        
    licStr=""
    lic=soup.find_all(class_='pv-certifications__summary-info pv-entity__summary-info pv-entity__summary-info--background-section pv-certifications__summary-info--has-extra-details')
    if not lic:
        appendData.append("NA")
    else:
        for item in lic:
            item2=item.getText()
            item2=item2.split('\n')
            for item3 in item2:
                if item3 not in ['Issuing authority','\n','Issued date and, if applicable, expiration date of the certification or license','Credential Identifier']:
                    licStr=licStr+str(item3)+" "
            licStr=licStr+" | "
        appendData.append(licStr)

    skills=soup.find_all(class_='pv-skill-category-entity__name-text t-16 t-black t-bold')
    skillStr=""
    if not skills:
        appendData.append("NA")
    else:
        for item in skills:
            item2=item.getText()
            item2=item2.split('\n')
            for item3 in item2:
                if item3 not in['\n','',' ']:
                    skillStr=skillStr+item3+" | "
        appendData.append(skillStr)
                
    
    accom=soup.find_all(class_='pv-accomplishments-block__content break-words')
    accomL=""
    if not accom:
        appendData.append("NA")
    else:
        for item in accom:
            item2=item.getText()
            item2=item2.split('\n')
            for item3 in item2:
                if item3 in ['Honors & Awards','Languages',"Language",'Test Scores','Test Score','Organization','Organizations','Project','Projects','Course','Courses']:
                    accomL=accomL+" | "+item3+": "
                else:
                    if item3 not in ['\n','',' ']:
                        accomL=accomL+item3+", "

        appendData.append(accomL)
        
    inter=soup.find_all(class_='pv-entity__summary-title-text')
    interL=""
    if not inter:
        appendData.append("NA")
    else:
        for item in inter:
            item2=item.getText()
            interL=interL+item2+" | "
            
        appendData.append(interL)
        
    
    with open('linkedinProfileData.csv', 'a') as csvFile:
        csvWriter = csv.writer(csvFile)
        csvWriter.writerow(appendData)
    
    #print(appendData)
    nextName=soup.find_all('span',class_='name',limit=5)
    nextLink=soup.find_all(class_="pv-browsemap-section__member ember-view",limit=5)
    if not nextLink:
        nextLink=soup.find_all(class_='ember-view pv-pymk-section__member',limit=5)
    #print(nextName,nextLink)
    for i in range(len(nextName)):
        nextName[i]=[nextName[i].getText(),"https://www.linkedin.com"+nextLink[i]['href']]
    nextNameCopy=nextName.copy()
    if len(nextName)==0:
        nextNameCopy="NA"
    else:
        for i in range(len(nextName)):
            if nextName[i] in nameList:
                nextNameCopy.remove(nextName[i])
    return (nextNameCopy)

In [4]:
#controls the running of the scraping loop
def control(driver,start,number=1,req=100):
    ind=number-1
    nameList=start
    while number<req:
        store=scrapeData(driver,nameList[ind][1],number,nameList)
        if store != "NA":
            for item in store:
                nameList.append(item)
            with open('dump.data', 'wb') as filehandle:
                # store the data as binary data stream
                pickle.dump(nameList, filehandle)
        ind+=1
        number+=1
        time.sleep(3)
    driver.close()
    driver.quit()

In [5]:
#login credentials
def login(start,email,passw,number=1,req=100):
    options=Options()
    driver = webdriver.Firefox(options=options ,executable_path='/usr/local/Cellar/geckodriver/0.26.0/bin/geckodriver')
    #time.sleep(60)
    driver.get('https://www.linkedin.com/uas/login')
    driver.find_element_by_name('session_key').send_keys(email)
    driver.find_element_by_name('session_password').send_keys(passw)
    time.sleep(5)
    try:
        driver.find_element_by_xpath("//button[@class='btn__primary--large from__button--floating mercado-button--primary']").click()
    except:
        driver.find_element_by_xpath("//button[@class='btn__primary--large from__button--floating']").click()
    #time.sleep(30)
    control(driver,start,number,req)

In [6]:
#initial run
def initialRun(startURL,startName,email,passw,req):
    createFile()
    start=[[startName,startURL]]
    login(start,email,passw,1,req)

In [7]:
#subsequent runs
def subsequentRun(email,passw,number,req):
    with open('dump.data', 'rb') as filehandle:
    # read the data as binary data stream
        nameList = pickle.load(filehandle)
    #print(len(nameList))
    login(nameList,email,passw,number,req)